In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-marketing-dataset/bank.csv
/kaggle/input/papers/Game_Theory_A_Very_Short_Introduction_----_(1_The_name_of_the_game).pdf


In [2]:
!pip install -U duckdb
!pip install llama-index-vector-stores-duckdb
!pip install llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 72.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 76.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.1.1
    Uninstalling duckdb-1.1.1:
      Successfully uninstalled duckdb-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import duckdb
conn = duckdb.connect('rag.duckdb')

In [4]:
conn.execute("""CREATE TABLE IF NOT EXISTS bank AS
SELECT * FROM read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')""")

In [5]:
conn.execute("""SHOW ALL TABLES""").fetch_df()

,database,schema,name,column_names,column_types,temporary
0,rag,main,bank,"[age, job, marital, education, default, balanc...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BOOLEAN, B...",False


In [6]:
conn.execute("""SELECT * FROM bank where duration <200 LIMIT 5""").fetchdf()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,31,blue-collar,married,secondary,False,2,True,False,unknown,20,may,182,2,-1,0,unknown,True
1,58,retired,married,primary,False,1241,False,False,cellular,18,aug,194,2,-1,0,unknown,True
2,55,blue-collar,married,secondary,False,196,False,False,cellular,21,aug,81,1,-1,0,unknown,True
3,52,technician,married,secondary,False,5303,True,False,unknown,19,oct,133,1,-1,0,unknown,True
4,33,services,married,secondary,False,3444,True,False,telephone,21,oct,144,1,91,4,failure,True


In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.core import StorageContext

In [8]:
import os
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

In [9]:
from huggingface_hub import login
login(token = '')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
llm = HuggingFaceLLM(tokenizer_name="meta-llama/Llama-3.2-1B",
    model_name="meta-llama/Llama-3.2-1B")

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [11]:
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 34.7 MB/s eta 0:00:00


In [12]:
!pip install llama-index-readers-file

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.0.1
    Uninstalling pypdf-5.0.1:
      Successfully uninstalled pypdf-5.0.1


In [13]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
Settings.llm = llm
Settings.chunk_size = 1024

In [15]:
documents = SimpleDirectoryReader('/kaggle/input/papers').load_data()

In [16]:
vector_store = DuckDBVectorStore(database_name = "rag.duckdb",table_name = "papers",persist_dir="./", embed_dim=1536)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index =VectorStoreIndex.from_documents(documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
index

In [32]:
query_engine= index.as_query_engine()
response = query_engine.query("What is Prisoner's dilemma in Game Theory?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [33]:
print(str(response.response))

1. The Prisoner's Dilemma is a game in game theory in which two individuals are placed in a situation where they must choose between cooperating or defecting. In this game, both individuals are in the same situation, and each has the opportunity to defect. The game is a zero-sum game, meaning that the sum of the two players' gains and losses is zero. If both players defect, then they both get a negative payoff. If both players cooperate, then they both get a positive payoff. 2. The Prisoner's Dilemma is a game in game theory in which two individuals are placed in a situation where they must choose between cooperating or defecting. In this game, both individuals are in the same situation, and each has the opportunity to defect. The game is a zero-sum game, meaning that the sum of the two players' gains and losses is zero. If both players cooperate, then they both get a positive payoff. If both players defect, then they both get a negative payoff. 3. The Prisoner's Dilemma is a game in g

In [34]:
print(str(response.get_formatted_sources()))

> Source (Doc id: d8769319-2b7d-4d90-b249-5feef7cd7469): An	alternative	response	is	to	argue	that	it	doesn’t	matter	what	is	rational	in	the	Prisoner’s
Dil...

> Source (Doc id: e884814c-0cf3-4d24-bd33-15d46586bbd6): 7.	Two	versions	of	the	Prisoner’s	Dilemma:	in	the	version	on	the	right,	
dove
	represents
giving	...
